In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
base_url="https://www.starwars.com/databank/"
names = pd.read_csv('./data/people.csv')

In [ ]:
data=[]

for name in names['name']:
    url=f"{base_url}{name.lower().replace(' ', '-')}"
    res=requests.get(url)
    soup=BeautifulSoup(res.content, 'html.parser')

    row=soup.find('div',class_='aspect')
    img=None
    if row is not None:
        img_element=row.find('img')
        if img_element is not None:
            img=img_element.get('data-src')
            if img is not None:
                img = img.replace('&width=320', '')
    
    row=soup.find('div',class_='content-bumper')
    if row is not None:
        row=row.find('div',class_='desc-sizer')
    desc=None
    if row is not None:
        desc_element=row.find('p',class_='desc')
        if desc_element is not None:
            desc=desc_element.get_text()
    data.append([name,img,desc])

df = pd.DataFrame(data, columns=['name', 'image', 'description'])

In [ ]:
df

In [ ]:
df.to_csv('../output/people-out.csv', index=False)


In [1]:
import os
from supabase import create_client,Client
from dotenv import load_dotenv

load_dotenv()

url: str = os.environ.get('SUPABASE_URL')
key: str = os.environ.get('SUPABASE_ANON_KEY')
supabase: Client = create_client(url, key)

In [ ]:
for index, row in df.iterrows():
    name = row['name']
    image = row['image']
    description = row['description']

    init = {
        'image': image,
        'description': description
    }

    data,count = supabase.table('people').update(init).eq('name',name).execute()
    print(data,count)
